In [1]:
import os 
import pandas as pd 

try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")

data = pd.read_csv("guillaume.txt", sep = ";")
data.info()

data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4646773 entries, 0 to 4646772
Data columns (total 23 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   ZIBZIN                    object
 1   IDAvisAutorisationCheque  int64 
 2   FlagImpaye                int64 
 3   Montant                   object
 4   DateTransaction           object
 5   CodeDecision              int64 
 6   VerifianceCPT1            int64 
 7   VerifianceCPT2            int64 
 8   VerifianceCPT3            int64 
 9   D2CB                      int64 
 10  ScoringFP1                object
 11  ScoringFP2                object
 12  ScoringFP3                object
 13  TauxImpNb_RB              object
 14  TauxImpNB_CPM             object
 15  EcartNumCheq              int64 
 16  NbrMagasin3J              int64 
 17  DiffDateTr1               object
 18  DiffDateTr2               object
 19  DiffDateTr3               object
 20  CA3TRetMtt                object
 21  CA3TR   

,ZIBZIN,IDAvisAutorisationCheque,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,...,TauxImpNb_RB,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure
0,A013010004908126703060931,78643044,0,20,2017-02-01 07:32:14,1,0,0,0,551,...,"37,186667890919111","52,076033757361408",0,1,4,4,4,20,0,27134
1,A013011306908024927155000,78643045,0,20,2017-02-01 07:43:37,1,0,0,0,551,...,"48,844716275908937","52,076033757361408",1,2,"1,7976851851851852",4,4,"28,609999999999999","8,6099999999999994",27817
2,A013010002908283134592527,78643046,0,"57,640000000000001",2017-02-01 07:47:38,1,0,0,0,549,...,"73,118279569892479","52,076033757361408",0,1,4,4,4,"57,640000000000001",0,28058
3,A011010002908105209831316,78643047,0,"54,289999999999999",2017-02-01 07:48:48,0,1,1,1,267,...,"110,05692599620494","53,554233554497365",0,1,4,4,4,"54,289999999999999",0,28128
4,A013010041908000125652029,78643048,0,"26,899999999999999",2017-02-01 08:13:27,1,0,0,0,549,...,"45,36831264567185","52,076033757361408",1,1,"1,9971064814814814",4,4,"59,149999999999999","32,25",29607


In [ ]:
#recode type of variable 
var_quanti = ["Montant","VerifianceCPT1", "VerifianceCPT2", "VerifianceCPT3","D2CB","ScoringFP1","ScoringFP2","ScoringFP3","TauxImpNb_RB","TauxImpNB_CPM","EcartNumCheq","NbrMagasin3J","DiffDateTr1", "DiffDateTr2","DiffDateTr3","CA3TRetMtt","CA3TR","Heure"]

var_quali = ["FlagImpaye" , "IDAvisAutorisationCheque" , "CodeDecision"]


for i in var_quanti :
    
    data[i] = data[i].replace(",", ".", regex= True).astype(float).round(0)

for i in var_quali :
    
    data[i] = data[i].astype(object)
    


In [ ]:
countTarget = data.FlagImpaye.value_counts()


In [46]:
import os 
import pandas as pd 
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd
import fastparquet
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import VarianceThreshold

try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")
    
train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')

Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "CodeDecision", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = train['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction","CodeDecision", 'Unnamed: 0'], axis = 1)
Ytest  = test.FlagImpaye

In [ ]:
Ytrain.FlagImpaye.value_counts()

In [3]:
from sklearn.utils import class_weight

classes = np.unique(Ytrain)
cw = class_weight.compute_class_weight(class_weight = 'balanced',classes =  np.unique(Ytrain),y= Ytrain)
weights = dict(zip(classes,cw))

In [64]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
Xtrain_norm = normalizer.transform(Xtrain)
Xtest_norm = normalizer.transform(Xtest)

C:\Users\Sam\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Normalizer was fitted without feature names
  warnings.warn(
C:\Users\Sam\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Normalizer was fitted without feature names
  warnings.warn(


In [65]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(class_weight = weights,max_depth=50, alpha=0.7, loss='deviance', learning_rate=0.09, n_estimators=1000, subsample=0.6,reg_alpha = 1.6, reg_lambda = 1.1,random_state=0).fit(Xtrain_norm, Ytrain)
pred = lgbm.predict(Xtest_norm)
   
ctest = confusion_matrix(Ytest, pred)
ftest = (2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0])) * 100
ftest

4.769882455495171

In [63]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier( LGBMClassifier(class_weight = weights,max_depth=50, alpha=0.7, loss='deviance', learning_rate=0.09, n_estimators=1000, subsample=0.6,reg_alpha = 1.6, reg_lambda = 1.1,random_state=0),n_estimators=150, random_state=0).fit(Xtrain, Ytrain)

pred = clf.predict(Xtest)
   
ctest = confusion_matrix(Ytest, pred)
ftest = (2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0])) * 100
ftest

KeyboardInterrupt: 

In [47]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(lgbm, prefit=True)
Xtrain_2 = Xtrain[Xtrain.columns[model.get_support(indices = True)]]
Xtest_2 = Xtest[Xtest.columns[model.get_support(indices = True)]]

In [48]:

lgbm2 = LGBMClassifier(class_weight = weights,max_depth=50, alpha=0.7, loss='deviance', learning_rate=0.09, n_estimators=1000, subsample=0.6,reg_alpha = 1.6, reg_lambda = 1.1,random_state=0).fit(Xtrain_2, Ytrain)
pred2 = lgbm2.predict(Xtest_2)
   
ctest2 = confusion_matrix(Ytest, pred2)
ftest2 = (2*ctest2[1,1]/(2*ctest2[1,1]+ctest2[0,1]+ctest2[1,0])) * 100
ftest2

5.170864558920251